In [33]:
import pandas as pd
from pandas import MultiIndex
import keras
import tensorflow as tf

In [34]:
train = pd.read_csv('train_test.hdf', 'train')

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


# Make TFRecords

In [35]:
def value_features(values):
    if type(values[0]) == np.float64:
        return tf.train.Feature(float_list=tf.train.FloatList(value=[v.astype(float) for v in values]))
    elif type(values[0]) == np.float:
        return tf.train.Feature(float_list=tf.train.FloatList(value=values))
    if type(values[0]) == np.int64:
        return tf.train.Feature(float_list=tf.train.Int64List(value=values))
    elif type(values[0]) == str:
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[v.encode() for v in values]))

def value_feature(value):
    if type(value) == np.float64:
        return tf.train.Feature(float_list=tf.train.FloatList(value=[value.astype(float)]))
    elif type(value) == np.float:
        return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    elif type(value) == np.int64:
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
    elif type(value) == int:
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
    elif type(value) == str:
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

In [36]:
info = pd.read_excel('Infos.xlsx').set_index(['id', 'study'])
info.sex.fillna('NaN', inplace=True)
info.self_size.fillna('NaN', inplace=True)
info.shirt_size.fillna('NaN', inplace=True)

In [37]:
writer = tf.python_io.TFRecordWriter('train.tfrecords')
for _, g in train.groupby(level=[0,1,2,3]):
    cur_info = info.loc[_[0], _[2]]
    feature = {'id': value_feature(_[0]), 'boardmac': value_feature(_[1]), 
               'study': value_feature(_[2]), 'period': value_feature(_[3])}
    feature.update({column: value_feature(cur_info[column])
               for column in ['sex', 'age', 'self_size', 'shirt_size', 'deodorant_left', 'deodorant_right']})
    feature.update({'sequence_length': value_feature(g.shape[0])})
    feature.update({column: value_features(g[column].values) for column in feature_columns})
    feature.update({'timestamp': value_features([dt.timestamp() for dt in g['datetime'].dt.to_pydatetime()])})
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    writer.write(example.SerializeToString())

ValueError: multiple levels only valid with MultiIndex

# Baseline Model

In [30]:
ts_columns = ['motion', 'temperature', 
              'sweat_10', 'sweat_11', 'sweat_12', 'sweat_13', 'sweat_14', 'sweat_15', 'sweat_16',
              'sweat_r0', 'sweat_r1', 'sweat_r2', 'sweat_r3', 'sweat_r4', 'sweat_r5', 'sweat_r6']

def _parse_function(example_proto):
    features = {
        'id': tf.FixedLenFeature((), tf.int64),
        'boardmac': tf.FixedLenFeature((), tf.string),
        'study': tf.FixedLenFeature((), tf.int64),
        'period': tf.FixedLenFeature((), tf.int64),
        'sex': tf.FixedLenFeature((), tf.string),
        'age': tf.FixedLenFeature((), tf.float32),
        'self_size': tf.FixedLenFeature((), tf.string),
        'shirt_size': tf.FixedLenFeature((), tf.string),
        'deodorant_left': tf.FixedLenFeature((), tf.int64),
        'deodorant_right': tf.FixedLenFeature((), tf.int64),
        'sequence_length': tf.FixedLenFeature((), tf.int64),
        'timestamp': tf.FixedLenSequenceFeature((), tf.float32, allow_missing=True)
    }
    features.update({column: tf.FixedLenSequenceFeature((), tf.float32, allow_missing=True) for column in ts_columns})
    return tf.parse_single_example(example_proto, features)

In [31]:
tf.reset_default_graph()

dataset = tf.data.TFRecordDataset('train.tfrecords').map(_parse_function).shuffle(256)
dataset = dataset.padded_batch(171, padded_shapes=dataset.output_shapes)
iterator = dataset.make_initializable_iterator()
example = iterator.get_next()

rnn_cell = keras.layers.LSTMCell(units=14)
rnn = keras.layers.RNN(rnn_cell, return_sequences=True)
t = example['timestamp']
t_diff = t[:,1:] - t[:, :-1]
x = tf.stack([t_diff, example['motion'][:, 1:] / 960.0], 2)
y_true = tf.stack([example[c][:, 1:] / 738600.0
                   for c in [c for c in ts_columns if c.startswith('sweat')]], 2)
y_pred = keras.layers.Dernn(x)
loss = tf.reduce_mean(tf.reduce_mean(keras.losses.mean_squared_error(y_true, y_pred), 1), 0)

opt = tf.train.AdamOptimizer()
opt_op = opt.minimize(loss)

saver = tf.train.Saver()

AttributeError: 'module' object has no attribute 'data'

In [ ]:
from tqdm import tqdm_notebook
pb = tqdm_notebook(range(1000))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'model/model.ckpt')
    
    for epoch in pb:
        sess.run(iterator.initializer)
        while True:
            try:
                cur_loss, _ = sess.run([loss, opt_op])
                pb.set_description("Current loss: %f" % cur_loss)
            except:
                break
        saver.save(sess, 'model/model.ckpt')

A Jupyter Widget

INFO:tensorflow:Restoring parameters from model/model.ckpt
